In [1]:
import torch
print("CUDA beschikbaar:", torch.cuda.is_available())
print("Aantal GPU's:", torch.cuda.device_count())
print(torch.version.cuda)  # Shows the CUDA version PyTorch was built with



CUDA beschikbaar: True
Aantal GPU's: 1
13.0


In [5]:
from ultralytics import YOLO
import os

# absolute path to your dataset
DATASET_DIR = r"C:\dev_projects_pc\uu-insect-project\object detection\insects\dataset_yolo_final"

# create YAML for YOLO
yaml_path = os.path.join(DATASET_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""
train: {os.path.join(DATASET_DIR, 'train', 'images')}
val: {os.path.join(DATASET_DIR, 'val', 'images')}
test: {os.path.join(DATASET_DIR, 'test', 'images')}

nc: 10  
names:
  0: 3031
  1: 6714
  2: 5953
  3: 5121
  4: 2351
  5: 2991
  6: 2392
  7: 6315
  8: 10896
  9: 7243
""")

# load model
model_choice = "yolov8m.pt"
model = YOLO(model_choice)

# train
results = model.train(
    data=yaml_path,
    epochs=60,
    imgsz=640,
    batch=16,
    patience=10,
    optimizer="AdamW",
    project="insect_yolo_results",
    device=0  # GPU
)

# evaluate on test set
metrics = model.val(split="test")

# make predictions
model.predict(source=os.path.join(DATASET_DIR, "test", "images"), save=True, conf=0.25)


Ultralytics 8.3.222  Python-3.10.0 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\dev_projects_pc\uu-insect-project\object detection\insects\dataset_yolo_final\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer

RuntimeError: No valid images found in C:\dev_projects_pc\uu-insect-project\object detection\insects\dataset_yolo_final\train\labels.cache. Images with incorrectly formatted labels are ignored. See https://docs.ultralytics.com/datasets for dataset formatting guidance.